In [ ]:
import time
import numpy as np
import pyscf

def restricted_HF_convE_and_DM(mol, conv_tol = 1e-10):
    """Restricted Hartree-Fock method, following Szabo and Ostlund.
    """
    start = time.time()

    n_a = mol.nelec[0]

    T = mol.intor("int1e_kin")
    Vnuc = mol.intor("int1e_nuc")
    hcore = T + Vnuc
    S = mol.intor("int1e_ovlp")
    h2e = mol.intor('int2e', aosym="s1") # most expensive, least intelligent way of doing this.

    convE = 10
    convDM = 10
    convComm = 10

    # we now implement checking convergence of both density matrix as well as energy.

    s, U = np.linalg.eigh(S)
    
    s_inv_sqrt = np.diag([x**(-0.5) for x in s])

    X = U @ s_inv_sqrt @ U.conj().T

    F0 = X.T @ hcore @ X
    _, C0 = np.linalg.eigh(F0)
    C = X @ C0
    P = 2 * C[:, :n_a] @ C[:, :n_a].T

    Eprev = None

    while convE > conv_tol or convDM > conv_tol or convComm > conv_tol:
        
        G = np.einsum('ij,klji->kl',P,h2e) - 0.5 * np.einsum('ij,kijl->kl',P,h2e)
        F = hcore + G

        E0 = 0.5 * np.einsum('ij,ij->',P,hcore+F)

        if Eprev is not None:
            convE = np.abs(E0 - Eprev)

        convComm = np.linalg.norm(F @ P @ S - S @ P @ F, 'fro')

        if convE < conv_tol and convDM < conv_tol and convComm < conv_tol:
            break
        
        Eprev = E0

        Fp = X.conj().T @ F @ X
        eps, Cp = np.linalg.eigh(Fp)
        C = X @ Cp

        Pnew = 2 * C[:,0:n_a] @ C[:,0:n_a].conj().T
        convDM = np.linalg.norm(Pnew-P)
        P = Pnew

    E0 = E0 + mol.energy_nuc()

    end = time.time()
    print(f"Converged in {end - start} seconds")

    return E0, P, F, C


In [56]:
import numpy as np
import scipy as sp
import pyscf
from pyscf import gto, scf

mol = gto.Mole()
mol.atom = """
H    0.0    0.0    0.0
H    1.0    0.0    0.0
H    2.0    0.0    0.0
H    3.0    0.0    0.0
"""
mol.basis = 'sto-3g'
mol.charge = 0
mol.spin = 0
mol.build()

In [57]:
soln = restricted_HF_convE_and_DM(mol)
S = mol.intor("int1e_ovlp")
my_energy = soln[0]
my_dm = soln[1]
my_fock = soln[2] 
my_orbitals = soln[3]
print("Energy: ", my_energy)
print("Density Matrix: ", my_dm)
print("Fock Matrix:", my_fock)
print("Commutator:", my_fock @ my_dm @ S - S @ my_dm @ my_fock)

Converged in 0.0518949031829834 seconds
Energy:  -2.098545936998037
Density Matrix:  [[ 0.71451376  0.64453946 -0.16511982 -0.398266  ]
 [ 0.64453946  0.65794533  0.06883671 -0.16511982]
 [-0.16511982  0.06883671  0.65794533  0.64453946]
 [-0.398266   -0.16511982  0.64453946  0.71451376]]
Fock Matrix: [[-0.24908988 -0.47921246 -0.15701381 -0.00295533]
 [-0.47921246 -0.35722517 -0.40907885 -0.15701381]
 [-0.15701381 -0.40907885 -0.35722517 -0.47921246]
 [-0.00295533 -0.15701381 -0.47921246 -0.24908988]]
Commutator: [[ 2.22044605e-16  1.11022302e-15  9.43689571e-16 -1.31838984e-16]
 [-8.88178420e-16 -2.22044605e-16  1.11022302e-16 -5.55111512e-16]
 [-6.66133815e-16 -1.11022302e-16  2.22044605e-16 -1.11022302e-16]
 [ 1.80411242e-16  2.22044605e-16  1.11022302e-16 -3.33066907e-16]]


In [58]:
fock_mo_basis = my_orbitals.T @ my_fock @ my_orbitals
print(fock_mo_basis)
orbital_energies = np.diag(fock_mo_basis)
print(orbital_energies)

[[-6.23341745e-01  2.23327506e-17 -4.45927675e-16  3.62972668e-16]
 [-2.19696977e-17 -3.82544171e-01  9.74630836e-17  1.48442861e-16]
 [-3.92237894e-16 -1.11247816e-16  2.96599949e-01 -2.32875313e-16]
 [-2.56128479e-16  1.67835325e-16 -3.61903610e-16  8.65755287e-01]]
[-0.62334175 -0.38254417  0.29659995  0.86575529]


In [60]:
## pyscf calculation

from pyscf import gto, scf, cc
from pyscf.cc.ccd import CCD

mf = scf.RHF(mol)
start = time.time()
mf.kernel()
end = time.time()

print(start - end, "seconds for RHF")


start = time.time()
mycc = cc.CCSD(mf)
mycc.kernel()
end = time.time()
print(start - end, "seconds for CCD")



converged SCF energy = -2.09854593699772
-1.057797908782959 seconds for RHF
E(CCSD) = -2.166379521639531  E_corr = -0.06783358464181333
-5.167368173599243 seconds for CCD


In [61]:
np.shape(mol.get_ovlp())

converged SCF energy = -2.09854593699772


(4, 4)

In [1]:
import itertools
import numpy as np
import scipy as sp

def ccsd(mol, reference, norb, nelec):
    # maximal difference between reference and final function is in 2 spots

    T = mol.intor("int1e_kin")
    Vnuc = mol.intor("int1e_nuc")
    hcore = T + Vnuc

    # assume restricted HF reference

    nalpha = nelec//2
    nbeta = nelec-nalpha

    E0 = reference[0]
    Pref = reference[1]
    Fock = reference[2]
    C = reference[3]

    # now need to get MOs

    orbital_energies = np.diag(C.T @ Fock @ C)

    occupied = C[:, :nalpha]
    unoccupied = C[:, nalpha:]

    # we would like to transform into the MO basis.

    F_mo = C.T @ Fock @ C

    hcore_mo = C.T @ hcore @ C

    h2e = mol.intor('int2e', aosym="s1")

    h2e_mo = np.einsum('pqrs, pi, qj, rk, sl -> ijkl', h2e, C, C, C, C, optimize = True)
    gpqrs = h2e_mo

    Lpqrs = np.zeros_like(gpqrs)
    for p in range(norb):
        for q in range(norb):
            for r in range(norb):
                for s in range(norb):
                    Lpqrs[p,q,r,s] = 2*gpqrs[p,q,r,s] - gpqrs[p,s,r,q]

    t1ai = np.zeros((norb, norb))
    t2ijab = np.zeros((norb,norb,norb,norb))

    # initial guess for t1 amplitudes should be zero
    # initial guess for t2 amplitudes should be the MP2 result, which happens to be:
    # (2(ia|jb) - (ib|ja))/(Dijab)

    for i in range(nalpha):
        for j in range(nalpha):
            for a in range(nalpha,norb):
                for b in range(nalpha,norb):
                    Dijab = orbital_energies[a] + orbital_energies[b] - orbital_energies[i] - orbital_energies[j]
                    t2ijab[i, j, a, b] = Lpqrs[i,a,j,b] / Dijab

    uijab = np.zeros_like(t2ijab)
    for i in range(nalpha):
        for j in range(nalpha):
            for a in range(nalpha,norb):
                for b in range(nalpha,norb):
                    uijab[i,j,a,b] = 2*t2ijab[i,j,a,b] - t2ijab[j,i,a,b]


    def generateX(t):
        return np.eye(norb,norb) - t

    def generateY(t):
        return np.eye(norb,norb) + t.T                    

    
    # need inactive Fock matrix; (10.8.28): Fmn = hmn + sum_i (2g_mnii - g_miin)

    
    converged = False
    en_CCSD = None
    while not converged:
        xmatrix = generateX(t1ai)
        ymatrix = generateY(t1ai)

        hcore_mo_t1 = np.einsum('pr,qs,rs->pq',xmatrix, ymatrix, hcore_mo)
        gpqrs_mo_t1 = np.einsum('pt,qu,rm,sn,tumn->pqrs', xmatrix, ymatrix,
                                   xmatrix, ymatrix, h2e_mo)

        occ = slice(None, nalpha)

        F_inactive_t1 = hcore_mo_t1 + 2*np.einsum('mnii->mn', gpqrs_mo_t1[:,:,occ,occ]) - np.einsum('miin->mn',gpqrs_mo_t1[:,occ,occ,:])
        Lpqrs_t1 = np.zeros_like(gpqrs_mo_t1)
        for p in range(norb):
            for q in range(norb):
                for r in range(norb):
                    for s in range(norb):
                        Lpqrs_t1[p,q,r,s] = 2*gpqrs_mo_t1[p,q,r,s] - gpqrs_mo_t1[p,s,r,q]
        
        omegaA1_ai = np.einsum('kicd,adkc->ai',uijab, gpqrs_mo_t1)
        omegaB1_ai = -np.einsum('klac, kilc->ai',uijab, gpqrs_mo_t1)
        omegaC1_ai = np.einsum('ikac,kc->ai',uijab, F_inactive_t1)
        omegaD1_ai = F_inactive_t1

        omegaA2_aibj = gpqrs_mo_t1 + np.einsum('ijcd,acbd->aibj',t2ijab, gpqrs_mo_t1)
        
        omegaB2_aibj = np.einsum('klab,kilj->aibj',t2ijab, gpqrs_mo_t1 + np.einsum('ijcd,kcld->kilj',t2ijab,gpqrs_mo_t1))
        
        omegaC2_aibj = -0.5*np.einsum('kjbc,kiac->aibj',t2ijab,gpqrs_mo_t1 - 
                                      0.5*np.einsum('liad,kdlc->kiac',t2ijab,gpqrs_mo_t1)) -np.einsum('kibc,kjac->aibj',t2ijab,gpqrs_mo_t1 - 0.5*np.einsum('ljad,kdlc->kjac',t2ijab,gpqrs_mo_t1))
        
        omegaD2_aibj = 0.5* np.einsum('jkbc,aikc->aibj',uijab,Lpqrs_t1 + 0.5*np.einsum('ilad,ldkc->aikc',uijab,Lpqrs_t1))
        
        omegaE2_aibj = np.einsum('ijac,bc->aibj',t2ijab,F_inactive_t1 - 
                                 np.einsum('klbd,ldkc->bc',uijab,gpqrs_mo_t1)) - np.einsum('ikab,kj->aibj',t2ijab,F_inactive_t1+np.einsum('ljcd,kdlc->kj',uijab,gpqrs_mo_t1)) 
        
        omega1_ai = omegaA1_ai + omegaB1_ai + omegaC1_ai + omegaD1_ai

        omega2_aibj = omegaA2_aibj + omegaB2_aibj + omegaC2_aibj + omegaD2_aibj + omegaE2_aibj + np.einsum('ijab->jiba', omegaC2_aibj + omegaD2_aibj + omegaE2_aibj)


        # now update t2 amplitudes

        for i in range(nalpha):
            for j in range(nalpha):
                for a in range(nalpha,norb):
                    for b in range(nalpha,norb):
                        t2ijab[i,j,a,b] = t2ijab[i,j,a,b] - omega2_aibj[a,i,b,j]/(orbital_energies[a]+orbital_energies[b]-orbital_energies[i]-orbital_energies[j])
        
        for i in range(nalpha):
            for a in range(nalpha,norb):
                t1ai[a,i] = t1ai[a,i] - omega1_ai[a,i]/(orbital_energies[a] - orbital_energies[i])


        en_CCSD_new = np.einsum('ijab,iajb->',t2ijab,Lpqrs) + np.einsum('ai,bj,iajb->',t1ai,t1ai,Lpqrs)

        if en_CCSD is not None:
            if np.abs(en_CCSD_new - en_CCSD) < 1e-6:
                en_CCSD = en_CCSD_new
                converged = True
        
        en_CCSD = en_CCSD_new
        print(en_CCSD)
    
    return en_CCSD, t1ai, t2ijab

In [74]:
## pyscf calculation

from pyscf import gto, scf, cc
from pyscf.cc.ccd import CCD

import numpy as np
import scipy as sp

mol = gto.Mole()
mol.atom = """
H    0.0    0.0    0.0
H    1.0    0.0    0.0
H    2.0    0.0    0.0
H    3.0    0.0    0.0
H    4.0    0.0    0.0
H    5.0    0.0    0.0
H    6.0    0.0    0.0
H    7.0    0.0    0.0
"""
mol.basis = 'sto-3g'
mol.charge = 0
mol.spin = 0
mol.build()

soln = restricted_HF_convE_and_DM(mol)

mf = scf.RHF(mol)
start = time.time()
mf.kernel()
end = time.time()

print(start - end, "seconds for RHF")


start = time.time()
mycc = cc.CCSD(mf)
mycc.kernel()
end = time.time()
print(start - end, "seconds for CCSD")

Converged in 0.17168879508972168 seconds
converged SCF energy = -4.17436981038916
-1.11114501953125 seconds for RHF
E(CCSD) = -4.306498896299036  E_corr = -0.1321290859098786
-1.8078680038452148 seconds for CCSD


In [76]:
ccsd(mol, soln, 8, 8)

0.04597247014840547
0.04992672110692782
0.051657773619321584
0.05269158484984926
0.05277684924816445
0.052807405509162936
0.0527371662002068
0.05269504287583571
0.0526562808441369
0.05263049717116683
0.052609303368509275
0.05259165005480074
0.05257580069933543
0.05256132826863466
0.05254814752900874
0.052536238405005986
0.05252566418812071
0.052516392462094746
0.05250838020752342
0.05250152595224966
0.05249571859793957
0.0524908310641069
0.05248674109948079
0.05248333178747014
0.0524804983538375
0.05247814794928517
0.0524762006699831
0.05247458835642145
0.05247325368870902
0.05247214872992223
0.05247123366308577


(0.05247123366308577,
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 1.61237740e-01,  9.51140865e-15,  7.07666133e-01,
          1.04598746e-14,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00],
        [ 3.12982285e-15,  7.30188136e-02, -3.37239498e-15,
          6.31363543e-01,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e